In [3]:
import matplotlib.pyplot as plt
import keras_ocr
import cv2
import math
import numpy as np
import os

In [2]:
def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2)/2)
    y_mid = int((y1 + y2)/2)
    return (x_mid, y_mid)

pipeline = keras_ocr.pipeline.Pipeline()

def inpaint_text(img_path, pipeline, correction_blur=1):
    # read image
    img = keras_ocr.tools.read(img_path)
    
    # generate (word, box) tuples 
    prediction_groups = pipeline.recognize([img])
    mask = np.zeros(img.shape[:2], dtype="uint8")
    
    prediction_groups_str = [[prediction_groups[0][i] for i in range(len(prediction_groups[0])) if not any(char.isdigit() for char in prediction_groups[0][i][0])]]
    prediction_groups_num = [[prediction_groups[0][i] for i in range(len(prediction_groups[0])) if any(char.isdigit() for char in prediction_groups[0][i][0])]]
    
    for box in prediction_groups[0]:
        x0, y0 = box[1][0]
        x1, y1 = box[1][1] 
        x2, y2 = box[1][2]
        x3, y3 = box[1][3] 
        
        x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
        x_mid1, y_mi1 = midpoint(x0, y0, x3, y3)
        
        thickness = int(math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 )) - correction_blur
        
        cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mi1), 255,    
        thickness)
        img = cv2.inpaint(img, mask, 7, cv2.INPAINT_NS)
                 
    return img, prediction_groups_num

Looking for /Users/michieldekoninck/.keras-ocr/craft_mlt_25k.h5
Looking for /Users/michieldekoninck/.keras-ocr/crnn_kurapan.h5


In [ ]:
import os
import random
from PIL import Image


pics = random.sample(os.listdir('../raw_data/img'), 15)

pics = os.listdir('../raw_data/img')

pics = [x for x in pics if int(x.split("_")[1])>2017]

print(len(pics))

In [ ]:
paths = ['../raw_data/img/'+x for x in pics]

In [ ]:
prediction_dict = {}

for pic in paths[1401:]:
    print(pic.split("/")[-1])
    
    image, prediction_groups_2 = inpaint_text(pic, pipeline)
    
    prediction_dict[pic] = prediction_groups_2
    
    image = cv2.resize(image, (300,150))

    #plt.figure(figsize=(30,10))
    #plt.imshow(image, cmap="gray")
    
    im = Image.fromarray(image)
    im.save(f'../raw_data/img_300/{pic.split("/")[-1]}')
        
    #plt.show()
    
    # im = Image.fromarray(image)
    # im.save(f'../raw_data/img_300/{pic.split("/")[-1]}')